# Load dependencies

In [107]:
import cobra
import libsbml
import lxml

cadaverine_directory = '/media/sf_Shared/Projects/Cadaverine_production/'
models_directory = '/media/sf_Shared/Systems_biology/Metabolic_models/'

In [108]:
iMsOB3b = cobra.io.load_json_model("/media/sf_Shared/Projects/Cadaverine_production/Models/" + 
                         "iMsOB3b_cadaverine.json")

In [109]:
iMsOB3b_original = cobra.io.load_json_model(models_directory + "iMsOB3b.json")

In [59]:
iMsOB3b.metabolites.lys_L_e

Metabolite identifier,lys_L_e
Name,
Memory address,0x07f7ca31b4ac8
Formula,None
Compartment,e
In 2 reaction(s),"CADVt, DM_lys_L_e"


# Check flux towards cadaverine with biomass as objective function: 

In [60]:
with iMsOB3b as model:
    model.optimize()
    print(model.reactions.PC.flux) #pyruvate carboxylase
    print(model.reactions.DAPDC.flux) #diaminopimelate decarboxylase
    print(model.reactions.LYSDC.flux) #lysine decarboxylase
    print(model.reactions.CADVt.flux) #cadaverine antiporter protein

0.0
0.031575378153261585
0.0
0.0


# Maximum cadaverine flux
Use cadaverine exchange reaction as an objective function

In [61]:
with iMsOB3b as model:
    model.objective = 'EX_15dap_e'
    model.reactions.DM_lys_L_e.bounds = (0, 1000)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(cadaverine_directory + 'simulations/' + 'cadaverine_antiport_objective_function'
                    + '.csv')
    model.summary()
    model.metabolites.lys_L_c.summary()

IN FLUXES       OUT FLUXES    OBJECTIVES
--------------  ------------  ---------------
h_e    999      co2_e  1e+03  EX_15dap_e  999
o2_e     1.75   h2o_e  1.75
ch4_e    0.875
PRODUCING REACTIONS -- L_lysine (lys_L_c)
-----------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%     999  CADVt     15dap_c + h_e + lys_L_e --> 15dap_e + h_c + lys...

CONSUMING REACTIONS -- L_lysine (lys_L_c)
-----------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  --------------------------------------------------
100%     999  LYSDC     h_c + lys_L_c --> 15dap_c + co2_c


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:6 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


**Note:** The problem with maximum flux towards cadaverine is that CADVt (cadaverine lysine antiporter) reaction depends on lys_L_e import creating futile cycle. The flux toward EX_15dap_e reaction (and thus cadaverine productivity) depends on amount of flux value for lys_L_e import.

## Add demand for DM_15dap_c

In [62]:
## Exchange reaction for 4hbD - EX_4hbD

from cobra import Model, Reaction, Metabolite
# Best practise: SBML compliant IDs

reaction = Reaction('DM_15dap_c')
reaction.name = 'Demand cadaverine'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [63]:
iMsOB3b.add_reaction(reaction)

In [64]:
iMsOB3b.reactions.get_by_id('DM_15dap_c').reaction = '15dap_c -->'

In [65]:
with iMsOB3b as model:
    model.objective = 'DM_15dap_c'
    model.reactions.DM_lys_L_e.bounds = (0, 1000)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(cadaverine_directory + 'simulations/' + 'cadaverine_antiport_objective_function_no3'
                    + '.csv')
    model.summary()
    model.metabolites.get_by_id("15dap_c").summary()

IN FLUXES     OUT FLUXES    OBJECTIVES
------------  ------------  -----------------
o2_e   22.3   h2o_e  25.7   DM_15dap_c  0.677
ch4_e  14.9   co2_e  11.5
h_e     3.05
no3_e   1.35
PRODUCING REACTIONS -- 1,5-Diaminopentane (15dap_c)
---------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  ---------------------------------
100%   0.677  LYSDC       h_c + lys_L_c --> 15dap_c + co2_c

CONSUMING REACTIONS -- 1,5-Diaminopentane (15dap_c)
---------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  ---------------------------------
100%   0.677  DM_15dap_c  15dap_c -->


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:6 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [66]:
with iMsOB3b as model:
    model.objective = 'DM_15dap_c'
    iMsOB3b.reactions.EX_nh4_e.bounds = (-1000, 0)
    model.reactions.DM_lys_L_e.bounds = (0, 1000)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(cadaverine_directory + 'simulations/' + 'cadaverine_antiport_objective_function_nh4'
                    + '.csv')
    model.summary()
    model.metabolites.get_by_id("15dap_c").summary()

IN FLUXES      OUT FLUXES    OBJECTIVES
-------------  ------------  ----------------
o2_e   22.4    h2o_e  25.5   DM_15dap_c  1.06
ch4_e  14.9    co2_e   9.58
nh4_e   2.13
h_e     0.532
PRODUCING REACTIONS -- 1,5-Diaminopentane (15dap_c)
---------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  ---------------------------------
100%    1.06  LYSDC       h_c + lys_L_c --> 15dap_c + co2_c

CONSUMING REACTIONS -- 1,5-Diaminopentane (15dap_c)
---------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  ---------------------------------
100%    1.06  DM_15dap_c  15dap_c -->


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


## Overexpression of genes (lysC) and (lysA)
ASPK (lysC) and DAPDC (lysA)

In [67]:
with iMsOB3b as model:
    solution = cobra.flux_analysis.pfba(model).fluxes
    print(solution["ASPK"])
    print(solution["DAPDC"])
    print(solution["PC"])

0.3232444289422262
0.031575378153261995
0.0


ASPK overexpression 2 fold:

In [68]:
with iMsOB3b as model:
    model.reactions.ASPK.bounds = ((0.32324442894222916*2), (0.32324442894222916*2))
    model.reactions.DM_lys_L_e.bounds = (0, 1000)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(cadaverine_directory + 'simulations/' + 'aspk_lysC_overexpression_no3'
                    + '.csv')
    model.summary()

IN FLUXES             OUT FLUXES    OBJECTIVES
--------------------  ------------  ----------------------
o2_e       22.4       h2o_e  26.4   Biomass_Mext...  0.123
ch4_e      14.9       co2_e   9.97
h_e         2.12
no3_e       1.2
pi_e        0.0857
k_e         0.0226
so4_e       0.00774
mg2_e       0.00101
fe2_e       0.000984
ca2_e       0.000607
cl_e        0.000607
na1_e       0.000499
mobd_e      0.000432
cu2_e       0.000405
mn2_e       0.000405
zn2_e       0.000405
cobalt2_e   2.69e-05


/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:6 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


DAPDC (lysA) 2 old:

In [69]:
with iMsOB3b as model:
    model.reactions.DAPDC.bounds = ((0.0315753781532617*2), (0.0315753781532617*2))
    model.reactions.DM_lys_L_e.bounds = (0, 1000)
    solution = cobra.flux_analysis.pfba(model).fluxes
    solution.to_csv(cadaverine_directory + 'simulations/' + 'dapdc_lysA_overexpression_no3'
                    + '.csv')
    model.summary()
    model.metabolites.get_by_id("15dap_c").summary()

/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:6 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


IN FLUXES          OUT FLUXES    OBJECTIVES
-----------------  ------------  ----------------------
o2_e    22.4       h2o_e  26.4   Biomass_Mext...  0.117
ch4_e   14.9       co2_e  10
h_e      3.25
no3_e    1.21
pi_e     0.0815
k_e      0.0215
so4_e    0.00736
mg2_e    0.000962
fe2_e    0.000936
cl_e     0.000577
ca2_e    0.000577
na1_e    0.000475
mobd_e   0.00041
cu2_e    0.000385
mn2_e    0.000385
zn2_e    0.000385
PRODUCING REACTIONS -- 1,5-Diaminopentane (15dap_c)
---------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  ---------------------------------
100%  0.0331  LYSDC       h_c + lys_L_c --> 15dap_c + co2_c

CONSUMING REACTIONS -- 1,5-Diaminopentane (15dap_c)
---------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  ---------------------------------
100%  0.0331  DM_15dap_c  15dap_c -->


## FVA

In [72]:
from cobra.flux_analysis import flux_variability_analysis

Loopsless FVA (NO3):

In [102]:
with iMsOB3b as model:
    model.reactions.CADVt.bounds = (0, 0)
    model.reactions.EX_15dap_e.bounds = (0, 0)
    model.reactions.DM_lys_L_e.bounds = (0, 0)
    FVA_loopless = flux_variability_analysis(model, model.reactions, fraction_of_optimum=1)
    FVA_loopless.to_csv(cadaverine_directory + "simulations/" + "FVA_loopless_no3.csv")

Loopsless FVA (NH4):

In [78]:
with iMsOB3b as model:
    model.reactions.CADVt.bounds = (0, 0)
    model.reactions.EX_15dap_e.bounds = (0, 0)
    model.reactions.DM_lys_L_e.bounds = (0, 0)
    model.reactions.EX_nh4_e.bounds = (-1000, 0)
    model.reactions.EX_no3_e.bounds = (0, 0)
    FVA_loopless = flux_variability_analysis(model, model.reactions, fraction_of_optimum=1)
    FVA_loopless.to_csv(cadaverine_directory + "simulations/" + "FVA_loopless_nh4.csv")

# MCMC sampling

NO3:

In [79]:
from cobra.test import create_test_model
from cobra.flux_analysis import sample
iMsOB3b_original.reactions.Biomass_Mextorquens_AM1_core.bounds = (0.1164735934, 1000) 
#95% of flux distribution

s = sample(iMsOB3b_original, 10000, seed=1)
s.head()

,EAOXRED,IMPC,HACD8i,PANTS,IMPD,OCPPH3,PGSA161,LCAD,PGSA160,PPAtex,...,UREA,SCYSDS,MOCDS,MCITD,NMNS,DHCYTCPEROX,PPK2,HAMR,HAORipp,ACSPW
0,0.000094,0.041193,0.002313,0.000027,0.014805,0.000027,0.000122,0.0,0.000121,0.0,...,0.0,0.0,0.0,0.0,0.0,0.002764,0.033220,0.000023,0.000175,0.110576
1,0.000093,0.040735,0.007907,0.000027,0.014640,0.000027,0.000121,0.0,0.000120,0.0,...,0.0,0.0,0.0,0.0,0.0,0.002934,0.051165,0.000414,0.003836,0.353358
2,0.000093,0.040615,0.026037,0.000026,0.014597,0.000026,0.000121,0.0,0.000120,0.0,...,0.0,0.0,0.0,0.0,0.0,0.006669,0.033147,0.000413,0.006691,0.368706
3,0.000091,0.039576,0.010087,0.000026,0.014223,0.000026,0.000118,0.0,0.000117,0.0,...,0.0,0.0,0.0,0.0,0.0,0.005464,0.127015,0.004714,0.005952,0.733484
4,0.000090,0.039493,0.002887,0.000026,0.014193,0.000026,0.000117,0.0,0.000116,0.0,...,0.0,0.0,0.0,0.0,0.0,0.005687,0.160756,0.004444,0.017407,0.764560


In [80]:
s.to_csv(cadaverine_directory + "simulations/" + "MCMC_sampling_original_no3.csv")

In [81]:
s.mean().to_csv(cadaverine_directory + "simulations/" + "MCMC_sampling_original_mean_no3.csv")

/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


NH4:

In [82]:
from cobra.test import create_test_model
from cobra.flux_analysis import sample
iMsOB3b_original.reactions.EX_no3_e.bounds = (0, 0)
iMsOB3b_original.reactions.EX_nh4_e.bounds = (-1000, 0)
iMsOB3b_original.reactions.Biomass_Mextorquens_AM1_core.bounds = (0.15871327233, 1000) 
#95% of flux distribution

s = sample(iMsOB3b_original, 10000, seed=1)
s.head()

,EAOXRED,IMPC,HACD8i,PANTS,IMPD,OCPPH3,PGSA161,LCAD,PGSA160,PPAtex,...,UREA,SCYSDS,MOCDS,MCITD,NMNS,DHCYTCPEROX,PPK2,HAMR,HAORipp,ACSPW
0,0.000123,0.053592,0.002141,0.000035,0.019261,0.000035,0.000159,0.0,0.000158,0.0,...,0.0,0.0,0.0,0.0,0.0,2.606578e-07,0.002009,0.009009,0.022094,1.146309
1,0.000123,0.053647,-0.000389,0.000035,0.019281,0.000035,0.000159,0.0,0.000158,0.0,...,0.0,0.0,0.0,0.0,0.0,7.612514e-05,0.011728,0.013345,0.156359,1.156918
2,0.000123,0.053729,-0.000917,0.000035,0.019310,0.000035,0.000160,0.0,0.000158,0.0,...,0.0,0.0,0.0,0.0,0.0,4.413000e-02,0.164569,0.029888,0.218318,1.059617
3,0.000123,0.053586,0.004375,0.000035,0.019258,0.000035,0.000159,0.0,0.000158,0.0,...,0.0,0.0,0.0,0.0,0.0,8.478634e-03,0.168738,0.015636,0.235540,1.091594
4,0.000123,0.053667,0.000361,0.000035,0.019288,0.000035,0.000159,0.0,0.000158,0.0,...,0.0,0.0,0.0,0.0,0.0,8.927307e-04,0.183211,0.004727,0.101502,1.087899


In [83]:
s.to_csv(cadaverine_directory + "simulations/" + "MCMC_sampling_original_nh4.csv")

In [84]:
s.mean().to_csv(cadaverine_directory + "simulations/" + "MCMC_sampling_original_mean_nh4.csv")

/home/ensakz/anaconda3/envs/GEM_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1 FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
